In [1]:
!pip install PyWavelets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

Matplotlib created a temporary cache directory at /tmp/matplotlib-5yf6bwq8 because the default path (/home/adelgado/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
files = glob.glob("shared-nvme/flujos_cepsa/procesa2-2021-0*.csv")

# Read CSV files
# Save it in a CSV or parquet??
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

# DataFrame with all the data
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
i = 0
wt='gaus1'
scaleMax = 12*24*7 # 5min*12=1h 1h*24=1d 1d*7=1week

# Plot every IP with more than 1000 points
for ip, filtered_df in combined_df.groupby("targetIP"):
    
    if filtered_df.shape[0] < 1000:
        continue
        
    i+=1
    xx = filtered_df.sort_values("tref_start", ascending=True)
    X = xx.tref_start
    Y = xx.bpsPhyRcv / xx.bpsPhyRcv.max()
    plt.figure()
    scales = np.arange(1,scaleMax,3) 
    coef, freqs=pywt.cwt(Y, scales, wt)

    file = f"descriptors/week/descriptor_{ip}_exp.npy"
    with open(file, 'wb') as f:
        np.save(file, coef)
    
    plt.matshow(coef) 
    plt.title(f"DWT({wt}) {ip}")
    plt.savefig(f"dwt/big_scales/{ip}_cwt_{wt}_{scaleMax}_plot.pdf")   # Guardar el gráfico en la carpeta "big_scales" dentro de Jupyter
    plt.close()

/tmp/ipykernel_682027/166979833.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.matshow(coef)
/tmp/ipykernel_682027/166979833.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [ ]:
descriptor = coef.sum(axis=1)
descriptor